# Import Libraries

In [ ]:
import sys
import os
from pathlib import Path

# Set working directory to project root, if not done already.
project_root = Path('/Users/raymondlow/Documents/talking-to-machines/ai-population').resolve()
os.chdir(project_root)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Set __package__ so that relative imports work.
__package__ = "ai_population.analysis"

import pandas as pd
import re
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from ai_population.src.market_signals_tiktok import (
    perform_tiktok_profile_search,
    perform_tiktok_profile_metadata_search,
    perform_tiktok_onboarding_interview,
)
from ai_population.src.utils import (
    perform_video_transcription,
    extract_llm_responses,
)

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

PROJECT_NAME = "market-signals-tiktok"
START_DATE = "12-01-2024"
END_DATE = "06-01-2025"

# Download profile metadata and posts for ground truth finfluencers and non-finfluencers

In [ ]:
perform_tiktok_profile_metadata_search(
    project_name=PROJECT_NAME,
    input_file_path="ground_truth_profile_list.csv",
    output_file_path="ground_truth_profile_metadata.csv",
)

In [ ]:
perform_tiktok_profile_search(
    project_name=PROJECT_NAME,
    input_file_path="ground_truth_profile_list.csv",
    output_file_path="ground_truth_profile_posts.csv",
    start_date=START_DATE,
    end_date=END_DATE,
)

# Download and transcribe videos from ground truth finfluencers and non-finfluencers

In [ ]:
perform_video_transcription(
    project_name=PROJECT_NAME,
    video_metadata_file="ground_truth_profile_posts.csv",
)

# Conduct Onboarding Interview for ground truth finfluencers and non-finfluencers

In [ ]:
perform_tiktok_onboarding_interview(
    project_name=PROJECT_NAME, 
    profile_metadata_file="ground_truth_profile_metadata.csv", 
    video_file="ground_truth_profile_posts.csv", 
    output_file="ground_truth_onboarding_results.csv",
)

# Preprocess Interview Data

In [ ]:
# def extract_llm_responses(text, substring_exclusion_list: list = []) -> pd.Series:
#     # Split the text by double newlines to separate different questions
#     questions_blocks = text.split("\n\n")
#     questions_blocks = [
#         block
#         for block in questions_blocks
#         if not any(substring in block for substring in substring_exclusion_list)
#     ]  # remove blocks containing stock recommendations

#     # Initialize lists to store the extracted data
#     questions_list = []
#     explanations_list = []
#     symbols_list = []
#     categories_list = []
#     speculations_list = []
#     values_list = []
#     response_list = []

#     # Define regex patterns for each field
#     question_pattern = r"\*\*question: (.*?)\*\*"
#     explanation_pattern = r"\*\*explanation: (.*?)\*\*"
#     symbol_pattern = r"\*\*symbol: (.*?)\*\*"
#     category_pattern = r"\*\*category: (.*?)\*\*"
#     speculation_pattern = r"\*\*speculation: (.*?)\*\*"
#     value_pattern = r"\*\*value: (.*?)\*\*"
#     response_pattern = r"\*\*response: (.*?)\*\*"

#     # Iterate through each question block and extract the fields
#     for block in questions_blocks:
#         question = re.search(question_pattern, block, re.DOTALL)
#         explanation = re.search(explanation_pattern, block, re.DOTALL)
#         symbol = re.search(symbol_pattern, block, re.DOTALL)
#         category = re.search(category_pattern, block, re.DOTALL)
#         speculation = re.search(speculation_pattern, block, re.DOTALL)
#         value = re.search(value_pattern, block, re.DOTALL)
#         response = re.search(response_pattern, block, re.DOTALL)

#         questions_list.append(question.group(1).replace("”", "") if question else None)
#         explanations_list.append(explanation.group(1) if explanation else None)
#         symbols_list.append(symbol.group(1) if symbol else None)
#         categories_list.append(category.group(1) if category else None)
#         speculations_list.append(speculation.group(1) if speculation else None)
#         values_list.append(value.group(1) if value else None)
#         response_list.append(response.group(1) if response else None)

#     # Create a DataFrame
#     data = {
#         "question": questions_list,
#         "explanation": explanations_list,
#         "symbol": symbols_list,
#         "category": categories_list,
#         "speculation": speculations_list,
#         "value": values_list,
#         "response": response_list,
#     }
#     df = pd.DataFrame(data)

#     # Flatten the DataFrame into a single Series
#     flattened_series = pd.Series()
#     for index, row in df.iterrows():
#         question_prefix = row["question"]
#         if row["explanation"]:
#             flattened_series[f"{question_prefix} - explanation"] = row["explanation"]
#         if row["symbol"]:
#             flattened_series[f"{question_prefix} - symbol"] = row["symbol"]
#         if row["category"]:
#             flattened_series[f"{question_prefix} - category"] = row["category"]
#         if row["speculation"]:
#             flattened_series[f"{question_prefix} - speculation"] = row["speculation"]
#         if row["value"]:
#             flattened_series[f"{question_prefix} - value"] = row["value"]
#         if row["response"]:
#             flattened_series[f"{question_prefix} - response"] = row["response"]

#     return flattened_series


extracted_results = identification_results["identification_llm_response"].apply(extract_llm_responses)
identification_results = pd.concat([identification_results, extracted_results], axis=1)
identification_results.head()

# Perform Identification Evaluation

In [ ]:
def calculate_metrics(df):
    # Convert "Is this a finfluencer? - category" column to binary values
    df['Is this a finfluencer? - category'] = df['Is this a finfluencer? - category'].map({'Yes': 1, 'No': 0})
    
    # Ensure there are no NaN values in the columns
    df = df.dropna(subset=['Finfluencer', 'Is this a finfluencer? - category'])
    
    # Extract the true labels and predicted labels
    y_true = df['Finfluencer']
    y_pred = df['Is this a finfluencer? - category']
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate macro-averaged F1 score
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    
    # Calculate AUC score
    auc = roc_auc_score(y_true, y_pred)
    
    return accuracy, macro_f1, auc

accuracy, macro_f1, auc = calculate_metrics(identification_results)
print(f'Accuracy: {accuracy:.5f}')
print(f'Macro-averaged F1 score: {macro_f1:.5f}')
print(f'AUC score: {auc:.5f}')